In [1]:
#Uses python3

import numpy as np
import pandas as pd 
import time

In [67]:
def coord(site):
    """get coordinate i of vector"""
    x = site // L
    y = site - x*L
    return (x,y)

def get(i):
    """fixin' boundary"""
    if i<0: return i
    else: return i % L
    
def get_neigh():
    """get neighbour's arr"""
    s = np.arange(L**2).reshape(L,L)
    nei = []
    for site in range(L*L):
        i,j = coord(site)
        nei += [s[get(i-1),get(j-1)],s[get(i-1),get(j)],s[get(i),get(j+1)],
                s[get(i+1),get(j+1)],s[get(i+1),get(j)],s[get(i),get(j-1)]]
    return np.array(nei, dtype=np.int32).reshape(L*L,6)

#=============================================================================

def gen_state():
    """generate random init. state with lenght L*L and q=[-1,1]"""
    return np.array([np.random.choice([-1,1]) for _ in range(L*L)], dtype=np.int32)

def mc_choice(dE,T,uni):
    """принимаем или не принимаем переворот спина?"""
    if dE > 0:
        return True
    elif uni <= np.exp(2*dE/T): 
        return True
    else:
        return False

def calc_dE(site, nei):
    """calculate dE"""
    summa = (
        nei[0]*nei[1]+nei[1]*nei[2]+
        nei[2]*nei[3]+nei[3]*nei[4]+
        nei[4]*nei[5]+nei[5]*nei[0]
    )
    return -site*summa
    
def step(s,nei,T,site,uni):
    """крутим 1 спин"""
    
    neigh = s[nei[site,:]]               
    dE = calc_dE(s[site], neigh)
    if mc_choice(dE,T,uni):
        s[site] *= -1


def mc_step(s,nei,T):
    """perform L*L flips for 1 MC step"""
    unis = np.random.uniform(size=L**2)
    sites = np.random.randint(L**2, size=L**2)
    for _ in range(L*L):
        step(s,nei,T, sites[_], unis[_])
        
        
def model(T,N_avg=10,N_mc=10,Relax=10):
    """Моделируем АТ"""
    E, M = [], []

    state = gen_state()
    nei = get_neigh()
    
    #relax $Relax times be4 AVG
    for __ in range(Relax):
            mc_step(state, nei, T)
    #AVG every $N_mc steps
    for _ in range(N_avg):
        for __ in range(N_mc):
            mc_step(state, nei, T)
        E += [calc_e(state)]
        M += [calc_m(state)]
    
    return E, M
        
#===========================================================================================

def create_mask():
    """маска в виде 3 под-решёток"""
    a = np.asarray([i % 3 for i in range(L)])
    return (a + a[:, None])%3

def calc_e(st):
    st = st.reshape(L,L)
    """calculate energy per site
        # expland state matrix"""
    a = np.concatenate((st[L-1].reshape(1,L), st, st[0].reshape(1,L)), axis=0) 
    b = np.concatenate((a[:,-1].reshape(L+2,1),a,a[:,0].reshape(L+2,1)), axis=1)
    return -np.sum(b[1:-1, 1:-1]*b[2:, 2:]*(b[2:, 1:-1]+b[1:-1, 2:]))/(L*L)  

def calc_ms(st):
    """magnetization"""
    st = st.reshape(L,L)
    msr = np.array([np.sum(st[mask==i]) for i in [0,1,2]])/(L*L)
    return np.sqrt(np.sum(msr*msr))



In [4]:
import cython
%reload_ext Cython

In [73]:
%%cython -a 
                            # --compile-args="-O2"

cimport cython
import numpy as np

cimport numpy as cnp
from numpy cimport npy_intp, npy_longdouble
from libcpp cimport bool
from libc.math cimport exp



@cython.boundscheck(False)
cdef int calc_e_jk(npy_intp site,
                   cnp.int32_t[::1] spins,          # the field
                   cnp.int32_t[:, :] neighbors):    # associations: neighbors[site, :] are local neighb sites
    """Calculate e_jk: the energy change for spins[site] -> *= -1"""
    
    cdef:
        int s_jk = spins[site]
        int summa = 0
        npy_intp prev = neighbors[site, 5]
        npy_intp curr
           
    
    for j in range(6):   
        curr = neighbors[site, j]
        summa += spins[curr] * spins[prev]
        prev = curr
    
    return -s_jk * summa


@cython.cdivision(True)
cdef bint mc_choice(int e_jk, double T, npy_longdouble uni):
    """принимаем или не принимаем переворот спина?"""
    cdef double r
    r = exp(2*e_jk/T)
    if e_jk > 0:
        return True
    elif uni <= r:
        return True
    else:
        return False


@cython.boundscheck(False) 
cdef void step(cnp.int32_t[::1] spins, cnp.int32_t[:, :] neigh, double T, npy_intp site, npy_longdouble uni):
    """крутим 1 спин"""

        
    cdef int L2, e_jk
    
    L2 = spins.shape[0]
    
    e_jk = calc_e_jk(site, spins, neigh)
    
    if mc_choice(e_jk, T, uni):
        spins[site] *= -1
        

@cython.boundscheck(False)
def mc_step(cnp.int32_t[::1] spins,
            cnp.int32_t[:, :] neighbors,
            double T):
    """perform L*L flips for 1 MC step"""
    
    cdef npy_intp num_steps = spins.shape[0]
    cdef int _
    
    cdef cnp.ndarray[double,
                ndim=1,
                negative_indices=False,
                mode='c'] unis = np.random.uniform(size=num_steps)
    cdef cnp.ndarray[npy_intp,
                ndim=1,
                negative_indices=False,
                mode='c'] sites = np.random.randint(num_steps, size=num_steps)
    
    for _ in range(num_steps):
        step(spins, neighbors, T, sites[_], unis[_])
        

In [71]:
global L, mask
L = 100
mask = create_mask()

np.random.seed(1)
s = gen_state(); nei = get_neigh()

%timeit [mc_step_c(s, nei, 1) for _ in range(10)]

6.1 ms ± 344 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [72]:
global L
L = 100

np.random.seed(1)
s = gen_state(); nei = get_neigh()
print(calc_e(s))

for _ in range(10):
    mc_step_c(s, nei, 1)
print(calc_e(s))

-0.0048
-1.084


In [61]:
# py     -- 2.16 s ± 52.6 ms per loop     -- original
# py v2  -- 3.02 s ± 81.8 ms per loop     -- tabulate neigh
# cy     -- 6.1 ms ± 344 µs per loop      -- cythonized & RNG 1 by 1

In [75]:
2.16/6.1*1000

354.09836065573774